<a href="https://colab.research.google.com/github/nqvftte/Painpoint_Solution_NguyenQuocViet/blob/main/Nguyen_Quoc_Viet_AI_Engineer_Intern_Assignments_FILUMAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install --upgrade --quiet llama-index-llms-together llama-index-readers-file
%pip install llama-index-retrievers-bm25

  Using cached llama_index_retrievers_bm25-0.5.2-py3-none-any.whl.metadata (740 bytes)
  Using cached bm25s-0.2.13-py3-none-any.whl.metadata (21 kB)
  Using cached PyStemmer-2.2.0.3-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
Using cached llama_index_retrievers_bm25-0.5.2-py3-none-any.whl (3.7 kB)
Using cached bm25s-0.2.13-py3-none-any.whl (54 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.3/669.3 kB 18.5 MB/s eta 0:00:00


In [ ]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
import os

if not os.environ.get("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = userdata.get("TOGETHER_API_KEY")

In [ ]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.llms import ChatMessage, MessageRole

llm = TogetherLLM(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
) #You should change your model name to be matched with NVIDIA_API_KEY



In [ ]:
from google.colab import drive
drive.mount('/content/driver/', force_remount=True)

Mounted at /content/driver/


In [ ]:
%cd "driver/MyDrive/TEST_FILUMAI"

/content/driver/MyDrive/TEST_FILUMAI


In [ ]:
%ls

context_features.json


In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import QueryFusionRetriever, VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.tools import QueryEngineTool

In [ ]:
import json
from llama_index.core import Document

# Load feature docs
def load_feature_docs(path):
    with open(path) as f:
        features = json.load(f)
    docs = []
    for feature in features:
        text = f"{feature['feature_name']}\n{feature['description']}\nCategory: {feature['category']}"
        docs.append(Document(text=text, metadata={"feature": feature['feature_name']}))
    return docs

docs = load_feature_docs("./context_features.json")

In [ ]:
print(docs[0])

Doc ID: a1348ea4-a657-4189-ac6e-0798f62fdc52
Text: Journeys Manage and visualize customer journeys and touchpoints
across various channels. Category: Voice of Customer (VoC)


In [ ]:
# Build hybrid retriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name = "BAAI/bge-base-en-v1.5" #"BAAI/bge-m3" for supporting multiple languages
)

index = VectorStoreIndex.from_documents(docs, embed_model=embed_model)
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3)
bm25_retriever = BM25Retriever.from_defaults(nodes=docs, similarity_top_k=3) # Changed from docs to nodes=docs
retriever = QueryFusionRetriever(
    [
        vector_retriever,
        bm25_retriever,
    ],
    llm=llm,
    num_queries=1,
    use_async=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

DEBUG:bm25s:Building index from IDs objects


In [ ]:
# Query engine to wrap in a tool
query_engine = RetrieverQueryEngine.from_args(llm=llm, retriever=retriever)
feature_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="feature_matcher",
    description="Matches a business pain point to Filum.ai features that can help solve it."
)

In [ ]:
from llama_index.core.agent.workflow import ReActAgent
Pain_Point_to_Solution_Agent = ReActAgent(llm=llm,
                   tools=[feature_tool],
                   verbose=False,
                   system_prompt=(
        "You are a helpful AI assistant that helps match business pain points to the most relevant Filum.ai features. "
        "Use the tool to retrieve the best matching features and explain how they help. "
        "Do not hallucinate features – always retrieve from the tool."
    ))

In [ ]:
response = await Pain_Point_to_Solution_Agent.run("We're overwhelmed by repeated questions in customer support.")

In [ ]:
print(response)

To address the issue of repeated questions in customer support, consider building unified customer profiles to identify common issues and provide personalized solutions. Additionally, analyze customer experiences across touchpoints to reveal patterns and sentiment around specific topics, enabling your support team to proactively address frequent questions and improve overall customer satisfaction.


In [ ]:
prompt_template = """You are an assistant that matches business pain points with exact Filum.ai features.

Select only from the following known features and categories:

- AI Inbox (AI Customer Service)
- Surveys (Voice of Customer)
- Experience (Insights)
- Customers (Customer 360)
...

Pain point: "{pain_point}"

Respond strictly in this format:

<Feature Name> (<Category>) – How it helps: <Short explanation of how this feature helps>"""


In [ ]:
pain_points = [
  "We're struggling to collect customer feedback consistently after a purchase.",
  "Our support agents are overwhelmed by the high volume of repetitive questions.",
  "We have no clear idea which customer touchpoints are causing the most frustration.",
  "It's difficult to get a single view of a customer's interaction history when they contact us.",
  "Manually analyzing thousands of open-ended survey responses for common themes is too time-consuming."
]

In [ ]:
for pain_point in pain_points:
  response = await Pain_Point_to_Solution_Agent.run(prompt_template.format(pain_point=pain_point))
  print("---------------------------------------------------------------------------------------------------------")
  print("- Question: {}\n- Answer: {}\n".format(pain_point, str(response)))

---------------------------------------------------------------------------------------------------------
- Question: We're struggling to collect customer feedback consistently after a purchase.
- Answer: Surveys (Voice of Customer) – How it helps: Collects customer feedback consistently after a purchase through automated surveys, providing valuable insights to improve customer experience.

---------------------------------------------------------------------------------------------------------
- Question: Our support agents are overwhelmed by the high volume of repetitive questions.
- Answer: AI Inbox (AI Customer Service) – How it helps: Automates responses to repetitive questions, reducing the workload of support agents and enabling them to focus on more complex issues.

---------------------------------------------------------------------------------------------------------
- Question: We have no clear idea which customer touchpoints are causing the most frustration.
- Answer: Expe